In [0]:
!pip install -r requirements.txt

In [0]:
import os
import numpy as np
import pyroomacoustics as pra
import matplotlib.pyplot as plt

from scipy.io import wavfile
from data_loader.preprocessing import *

In [0]:
import tempfile
import os
import boto3

s3 = boto3.resource('s3')

In [0]:
import pyroomacoustics as pra
from pyroomacoustics.directivities import (
    DirectivityPattern,
    DirectionVector,
    CardioidFamily,
)
dir_obj = CardioidFamily(
    orientation=DirectionVector(azimuth=0, colatitude=180, degrees=True),
    pattern_enum=DirectivityPattern.HYPERCARDIOID,
)

In [0]:
# 学習データの作成

'''
測位領域：3m x 4m
データ数1000: 30 x 40 = 1200(10cmおき)
データ数10000: 117 x 157 = 18369(2.5cmおき)
データ数100000: 293 x 393 = 115149(1cmおき)
'''

## channel: 1

In [0]:
# 測位用(5cmおき: データ数4941)

X_train = []
y_train = []

# お部屋の形決め
room_dim = [11.025, 3.7, 3.98]

#お部屋の素材決め
m = pra.make_materials(
    ceiling="hard_surface",
    floor="linoleum_on_concrete",
    east="rough_concrete",
    west="rough_concrete",
    north="rough_concrete",
    south="blinds_half_open",
)

room = pra.ShoeBox(
    room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
)

mic_loc = np.array([[], [], []])

for i in range(61*81):
    mic_loc = np.c_[mic_loc, [(i//61)/20 + 4.6, (i%61)/20 + 0.4, 1]]
    
room.add_microphone(mic_loc)

_, audio = wavfile.read("/dbfs/mnt/mnt_wg3-1/mita/wav/chirp16k_19k_1.25s.wav") # S3に適した形に書き換え

# 音源ごとに座標情報を与え、`room`に追加していきます。
# オプションで delay を追加することもできます。
room.add_source([3.6, 2.2, 2.29], signal=audio, delay=0.2, directivity=dir_obj)

# S/N 比
SNR = 5

#シミュレーション
room.simulate(SNR)

simulation_data = room.mic_array.signals

for i in range(simulation_data.shape[0]):
    
    X = simulation_data[i]
    snd = 0.5 * X / np.max(X)

    chirp = ChirpMaker(16000, 19000, 0.02, 1)
    index_sync = 0.2

    train = GetGSP(chirp, snd, index_sync, True, n=1)   # 実測の時はTrueを変える．
    
    # print(X_train.shape, train.shape)
    X_train.append(train)
    y_train.append([[(i%61)/20, (i//61)/20] for _ in range(train.shape[0])])

    print("\r"+f"{i+1} samples have been processed.",end="")

# 最後に numpy 配列に変換
X_train = np.vstack(X_train)
y_train = np.vstack(y_train)

print("\n")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train = X_train.reshape(-1, 1, 4800).astype("float32")
y_train = y_train.reshape(-1, 2).astype("float32")

with tempfile.TemporaryFile() as temp:
    np.savez(temp, X=X_train, pos=y_train)
    temp.seek(0)
    res = s3.Object(bucket_name="wg3-1", key="mita/simulation_data/train_4941.npz").upload_fileobj(temp)

# # 保存するのはX, posの2変数
# np.savez("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/train_4941.npz", X=X_train, pos=y_train)

plt.figure(figsize=(10, 10))
plt.scatter(y_train[:, 0], y_train[:, 1], s=1)
plt.scatter([1.8], [-1.0], color="black", marker="*", s=2, label="speaker")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Training data")
plt.xlim(-0.4, 3.3)
plt.ylim(4.2, -1.2)
plt.legend()
plt.savefig("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/train_4941.png")


In [0]:
# 測位用(25cmおき: データ数221)

X_train = []
y_train = []

# お部屋の形決め
room_dim = [11.025, 3.7, 3.98]

#お部屋の素材決め
m = pra.make_materials(
    ceiling="hard_surface",
    floor="linoleum_on_concrete",
    east="rough_concrete",
    west="rough_concrete",
    north="rough_concrete",
    south="blinds_half_open",
)

room = pra.ShoeBox(
    room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
)

mic_loc = np.array([[], [], []])

for i in range(13 * 17):
    mic_loc = np.c_[mic_loc, [(i//13)/4 + 4.6, (i%13)/4 + 0.4, 1]]
    
room.add_microphone(mic_loc)

_, audio = wavfile.read("/dbfs/mnt/mnt_wg3-1/mita/wav/chirp16k_19k_1.25s.wav") # S3に適した形に書き換え

# 音源ごとに座標情報を与え、`room`に追加していきます。
# オプションで delay を追加することもできます。
room.add_source([3.6, 2.2, 2.29], signal=audio, delay=0.2, directivity=dir_obj)

# S/N 比
SNR = 5

#シミュレーション
room.simulate(SNR)

simulation_data = room.mic_array.signals

for i in range(simulation_data.shape[0]):
    
    X = simulation_data[i]
    snd = 0.5 * X / np.max(X)

    chirp = ChirpMaker(16000, 19000, 0.02, 1)
    index_sync = 0.2

    train = GetGSP(chirp, snd, index_sync, True, n=1)   # 実測の時はTrueを変える．
    
    # print(X_train.shape, train.shape)
    X_train.append(train)
    y_train.append([[(i%13)/4, (i//13)/4] for _ in range(train.shape[0])])

    print("\r"+f"{i+1} samples have been processed.",end="")

# 最後に numpy 配列に変換
X_train = np.vstack(X_train)
y_train = np.vstack(y_train)

print("\n")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train = X_train.reshape(-1, 1, 4800).astype("float32")
y_train = y_train.reshape(-1, 2).astype("float32")

# 保存するのはX, posの2変数
with tempfile.TemporaryFile() as temp:
    np.savez(temp, X=X_train, pos=y_train)
    temp.seek(0)
    res = s3.Object(bucket_name="wg3-1", key="mita/simulation_data/train_221.npz").upload_fileobj(temp)


plt.figure(figsize=(10, 10))
plt.scatter(y_train[:, 0], y_train[:, 1], s=1)
plt.scatter([1.8], [-1.0], color="black", marker="*", s=2, label="speaker")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Training data")
plt.xlim(-0.4, 3.3)
plt.ylim(4.2, -1.2)
plt.legend()
plt.savefig("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/train_221.png")


In [0]:
# (25cmおき: データ数165)

X_train = []
y_train = []
pos_train = []

for i in range(13*17):

    # お部屋の形決め
    room_dim = [11.025, 3.7, 3.98]

    #お部屋の素材決め
    m = pra.make_materials(
        ceiling="hard_surface",
        floor="linoleum_on_concrete",
        east="rough_concrete",
        west="rough_concrete",
        north="rough_concrete",
        south="blinds_half_open",
    )

    room = pra.ShoeBox(
        room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
    )
    mic_loc = np.array([[], [], []])

    # 左右2点前後2点ヒントの距離ヒントの方向ヒントの座標
    mic_loc = np.c_[mic_loc, [(i//13)/4 + 4.6, (i%13)/4 + 0.4, 1]]
    room.add_microphone(mic_loc)

    _, audio = wavfile.read("/dbfs/mnt/mnt_wg3-1/mita/wav/chirp16k_19k_1.25s.wav") # S3に適した形に書き換え

    # 音源ごとに座標情報を与え、`room`に追加していきます。
    # オプションで delay を追加することもできます。
    room.add_source([3.6, 2.2, 2.29], signal=audio, delay=0.2, directivity=dir_obj)

    # S/N 比
    SNR = 5

    #シミュレーション
    room.simulate(SNR)

    simulation_data = room.mic_array.signals

    
    X1 = simulation_data[0]
    snd1 = 0.5 * X1 / np.max(X1)

    chirp = ChirpMaker(16000, 19000, 0.02, 1)
    index_sync = 0.2

    train1 = GetGSP(chirp, snd1, index_sync, True, n=2)

    # print(X_train.shape, train.shape)
    for j in range(len(train1)):
        X_train.append([train1[j]])
        y_train.append([train1[j]])
        pos_train.append([(i%13)/4, (i//13)/4])

    print("\r"+f"{i+1} samples have been processed.",end="")

# 最後に numpy 配列に変換
X_train = np.array(X_train).astype("float32")
y_train = np.array(y_train).astype("float32")
pos_train = np.array(pos_train).astype("float32")

print("\n")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("pos_train shape:", pos_train.shape)

# 保存するのはX, y, posの3変数
np.savez("../data/train_2_data221.npz", X=X_train, y=y_train, pos=pos_train)


plt.figure(figsize=(10, 10))
plt.scatter(pos_train[:, 0], pos_train[:, 1], s=1)
plt.scatter([1.8], [-1.0], color="black", marker="*", s=2, label="speaker")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Training data")
plt.xlim(-0.4, 3.3)
plt.ylim(4.2, -1.2)
plt.legend()
plt.savefig("../data/fig/train_2_data221.png")

In [0]:
plt.plot(X_train[0])
plt.show()

In [0]:
# 波形復元用(10cmおき: データ数1200)

X_train = []
y_train = []
pos_train = []


# お部屋の形決め
room_dim = [11.025, 3.7, 3.98]

#お部屋の素材決め
m = pra.make_materials(
    ceiling="hard_surface",
    floor="linoleum_on_concrete",
    east="rough_concrete",
    west="rough_concrete",
    north="rough_concrete",
    south="blinds_half_open",
)

room = pra.ShoeBox(
    room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
)
mic_loc = np.array([[], [], []])

# 
for i in range(30*40): 
    mic_loc = np.c_[mic_loc, [(i//30)/10 + 4.65, (i%30)/10 + 0.45, 1]]
    
room.add_microphone(mic_loc)

_, audio = wavfile.read("/dbfs/mnt/mnt_wg3-1/mita/wav/chirp16k_19k_1.25s.wav") # S3に適した形に書き換え

# 音源ごとに座標情報を与え、`room`に追加していきます。
# オプションで delay を追加することもできます。
room.add_source([3.6, 2.2, 2.29], signal=audio, delay=0.2, directivity=dir_obj)

# S/N 比
SNR = 5

#シミュレーション
room.simulate(SNR)

simulation_data = room.mic_array.signals

for i in range(30*40):    
    X = simulation_data[i]
    snd = 0.5 * X / np.max(X)

    chirp = ChirpMaker(16000, 19000, 0.02, 1)
    index_sync = 0.2

    train1 = GetGSP(chirp, snd, index_sync, True, n=1)
    
    # print(X_train.shape, train.shape)
    X_train.append(train1)
    y_train.append(train1)
    pos_train.append([(i%30)/10+0.05, (i//30)/10+0.05])

    print("\r"+f"{i+1} samples have been processed.",end="")

# 最後に numpy 配列に変換
X_train = np.squeeze(X_train).astype("float32")
y_train = np.array(y_train).astype("float32")
pos_train = np.array(pos_train).astype("float32")

print("\n")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("pos_train shape:", pos_train.shape)

# 保存するのはX, y, posの3変数
np.savez("../data/train_data1000.npz", X=X_train, y=y_train, pos=pos_train)


plt.figure(figsize=(10, 10))
plt.scatter(pos_train[:, 0], pos_train[:, 1], s=1)
plt.scatter([1.8], [-1.0], color="black", marker="*", s=2, label="speaker")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Training data")
plt.xlim(-0.4, 3.3)
plt.ylim(4.2, -1.2)
plt.legend()
plt.savefig("../data/fig/train_data1000.png")

In [0]:
# Autoencoder用のデータ収集

X_train = []
y_train = []
pos_train = []

# お部屋の形決め
room_dim = [11.025, 3.7, 3.98]

#お部屋の素材決め
m = pra.make_materials(
    ceiling="hard_surface",
    floor="linoleum_on_concrete",
    east="rough_concrete",
    west="rough_concrete",
    north="rough_concrete",
    south="blinds_half_open",
)

room = pra.ShoeBox(
    room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
)

train_num = 100000
np.random.seed(2025)
x_pos = np.random.uniform(0, 4, train_num)
y_pos = np.random.uniform(0, 3, train_num)

for i in range(train_num):

    # お部屋の形決め
    room_dim = [11.025, 3.7, 3.98]

    #お部屋の素材決め
    m = pra.make_materials(
        ceiling="hard_surface",
        floor="linoleum_on_concrete",
        east="rough_concrete",
        west="rough_concrete",
        north="rough_concrete",
        south="blinds_half_open",
    )

    room = pra.ShoeBox(
        room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
    )

    mic_loc = np.array([[], [], []])
    mic_loc = np.c_[mic_loc, [x_pos[i] + 4.6, y_pos[i] + 0.4, 1]]
    room.add_microphone(mic_loc)

    _, audio = wavfile.read("/dbfs/mnt/mnt_wg3-1/mita/wav/chirp16k_19k_1.25s.wav") # S3に適した形に書き換え

    # 音源ごとに座標情報を与え、`room`に追加していきます。
    # オプションで delay を追加することもできます。
    room.add_source([3.6, 2.2, 2.29], signal=audio, delay=0.2, directivity=dir_obj)

    # S/N 比
    SNR = 5

    #シミュレーション
    room.simulate(SNR)

    simulation_data = room.mic_array.signals

        
    X1 = simulation_data[0]
    snd1 = 0.5 * X1 / np.max(X1)

    chirp = ChirpMaker(16000, 19000, 0.02, 1)
    index_sync = 0.2

    train1 = GetGSP(chirp, snd1, index_sync, True, n=1)
    
    # print(X_train.shape, train.shape)
    X_train.append(train1)
    y_train.append(train1)
    pos_train.append([y_pos[i], x_pos[i]])

    print("\r"+f"{i+1} samples have been processed.",end="")

# 最後に numpy 配列に変換
X_train = np.array(X_train).astype("float32")
y_train = np.array(y_train).astype("float32")
pos_train = np.vstack(pos_train).astype("float32")

print("\n")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("pos_train shape:", pos_train.shape)

# 保存するのはX, posの2変数
with tempfile.TemporaryFile() as temp:
    np.savez(temp, X=X_train, pos=pos_train)
    temp.seek(0)
    res = s3.Object(bucket_name="wg3-1", key="mita/simulation_data/train_random_100000.npz").upload_fileobj(temp)

plt.figure(figsize=(10, 10))
plt.scatter(pos_train[:, 0], pos_train[:, 1], s=1)
plt.scatter([1.8], [-1.0], color="black", marker="*", s=2, label="speaker")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("train data")
plt.xlim(-0.4, 3.3)
plt.ylim(4.2, -1.2)
plt.legend()
plt.savefig("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/train_random_100000.png")

## test

In [0]:
# テストデータの作成(1ch)

X_test = []
y_test = []
pos_test = []

# お部屋の形決め
room_dim = [11.025, 3.7, 3.98]

#お部屋の素材決め
m = pra.make_materials(
    ceiling="hard_surface",
    floor="linoleum_on_concrete",
    east="rough_concrete",
    west="rough_concrete",
    north="rough_concrete",
    south="blinds_half_open",
)

room = pra.ShoeBox(
    room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
)

mic_loc = np.array([[], [], []])

test_num = 1000
np.random.seed(2024)
x_pos = np.random.uniform(0, 4, test_num)
y_pos = np.random.uniform(0, 3, test_num)
for i in range(test_num):
    mic_loc = np.c_[mic_loc, [x_pos[i] + 4.6, y_pos[i] + 0.4, 1]]
room.add_microphone(mic_loc)

_, audio = wavfile.read("/dbfs/mnt/mnt_wg3-1/mita/wav/chirp16k_19k_1.25s.wav") # S3に適した形に書き換え

# 音源ごとに座標情報を与え、`room`に追加していきます。
# オプションで delay を追加することもできます。
room.add_source([3.6, 2.2, 2.29], signal=audio, delay=0.2, directivity=dir_obj)

# S/N 比
SNR = 5

#シミュレーション
room.simulate(SNR)

simulation_data = room.mic_array.signals

for i in range(simulation_data.shape[0]):
    
    X1 = simulation_data[i]
    snd1 = 0.5 * X1 / np.max(X1)

    chirp = ChirpMaker(16000, 19000, 0.02, 1)
    index_sync = 0.2

    test1 = GetGSP(chirp, snd1, index_sync, True, n=1)
    
    # print(X_train.shape, train.shape)
    X_test.append(test1)
    y_test.append(test1)
    pos_test.append([y_pos[i], x_pos[i]])

    print("\r"+f"{i+1} samples have been processed.",end="")

# 最後に numpy 配列に変換
X_test = np.array(X_test).astype("float32")
y_test = np.array(y_test).astype("float32")
pos_test = np.vstack(pos_test).astype("float32")

print("\n")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("pos_test shape:", pos_test.shape)

# 保存するのはX, posの2変数
with tempfile.TemporaryFile() as temp:
    np.savez(temp, X=X_test, pos=pos_test)
    temp.seek(0)
    res = s3.Object(bucket_name="wg3-1", key="mita/simulation_data/test_1000.npz").upload_fileobj(temp)

plt.figure(figsize=(10, 10))
plt.scatter(pos_test[:, 0], pos_test[:, 1], s=1)
plt.scatter([1.8], [-1.0], color="black", marker="*", s=2, label="speaker")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Test data")
plt.xlim(-0.4, 3.3)
plt.ylim(4.2, -1.2)
plt.legend()
plt.savefig("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/test_1000.png")

In [0]:
# テストデータの作成(1ch), 480samples

X_test = []
y_test = []
pos_test = []

# お部屋の形決め
room_dim = [11.025, 3.7, 3.98]

#お部屋の素材決め
m = pra.make_materials(
    ceiling="hard_surface",
    floor="linoleum_on_concrete",
    east="rough_concrete",
    west="rough_concrete",
    north="rough_concrete",
    south="blinds_half_open",
)

room = pra.ShoeBox(
    room_dim, fs=48000, materials=m, max_order=10, sigma2_awgn=0.3
)

mic_loc = np.array([[], [], []])

test_num = 1000
np.random.seed(2024)
x_pos = np.random.uniform(0, 4, test_num)
y_pos = np.random.uniform(0, 3, test_num)
for i in range(test_num):
    mic_loc = np.c_[mic_loc, [x_pos[i] + 4.6, y_pos[i] + 0.4, 1]]
room.add_microphone(mic_loc)

_, audio = wavfile.read("/dbfs/mnt/mnt_wg3-1/mita/wav/chirp16k_19k_1.25s.wav") # S3に適した形に書き換え

# 音源ごとに座標情報を与え、`room`に追加していきます。
# オプションで delay を追加することもできます。
room.add_source([3.6, 2.2, 2.29], signal=audio, delay=0.2, directivity=dir_obj)

# S/N 比
SNR = 5

#シミュレーション
room.simulate(SNR)

simulation_data = room.mic_array.signals

for i in range(simulation_data.shape[0]):
    
    X1 = simulation_data[i]
    snd1 = 0.5 * X1 / np.max(X1)

    chirp = ChirpMaker(16000, 19000, 0.02, 1)
    index_sync = 0.2

    test1 = GetGSP(chirp, snd1, index_sync, True, n=1)
    
    # print(X_train.shape, train.shape)
    X_test.append(test1)
    y_test.append(test1)
    pos_test.append([y_pos[i], x_pos[i]])

    print("\r"+f"{i+1} samples have been processed.",end="")

# 最後に numpy 配列に変換
X_test = np.array(X_test).astype("float32")
y_test = np.array(y_test).astype("float32")
pos_test = np.vstack(pos_test).astype("float32")

print("\n")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("pos_test shape:", pos_test.shape)

# 保存するのはX, posの2変数
with tempfile.TemporaryFile() as temp:
    np.savez(temp, X=X_test, pos=pos_test)
    temp.seek(0)
    res = s3.Object(bucket_name="wg3-1", key="mita/simulation_data/test_random_1000.npz").upload_fileobj(temp)

plt.figure(figsize=(10, 10))
plt.scatter(pos_test[:, 0], pos_test[:, 1], s=1)
plt.scatter([1.8], [-1.0], color="black", marker="*", s=2, label="speaker")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.title("Test data")
plt.xlim(-0.4, 3.3)
plt.ylim(4.2, -1.2)
plt.legend()
plt.savefig("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/test_random_1000.png")

## 0/1のデータセットの作成

In [0]:
data = np.load("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/train_random_100000.npz")
X_train = data["X"]
pos_train = data["pos"]

new_data = []

for i in range(len(X_train)):
    new = np.zeros((1, 480))
    pak = np.array(PeakDetector(X_train[i].squeeze(), 2, 20, 1.6))
    for j in pak[:, 0]:
        new[0][int(j)] = 1
    new_data.append(new)
    print(f"{i} samples has processed.", end="\r")


new_data = np.array(new_data)
print(new_data.shape, pos_train.shape)

# 保存するのはX, posの2変数
with tempfile.TemporaryFile() as temp:
    np.savez(temp, X=new_data, pos=pos_train)
    temp.seek(0)
    res = s3.Object(bucket_name="wg3-1", key="mita/simulation_data/train_random_100000_peak.npz").upload_fileobj(temp)


In [0]:
data = np.load("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/test_random_1000.npz")
X_test = data["X"]
pos_test = data["pos"]

new_data = []

for i in range(len(X_test)):
    new = np.zeros((1, 480))
    pak = np.array(PeakDetector(X_test[i].squeeze(), 2, 20, 1.6))
    for j in pak[:, 0]:
        new[0][int(j)] = 1
    new_data.append(new)
    print(f"{i} samples has processed.", end="\r")


new_data = np.array(new_data)
print(new_data.shape, pos_test.shape)

# 保存するのはX, posの2変数
with tempfile.TemporaryFile() as temp:
    np.savez(temp, X=new_data, pos=pos_test)
    temp.seek(0)
    res = s3.Object(bucket_name="wg3-1", key="mita/simulation_data/test_1000_peak.npz").upload_fileobj(temp)


In [0]:
data = np.load("/dbfs/mnt/mnt_wg3-1/mita/simulation_data/test_1000_peak.npz")
X = data["X"]
plt.plot(X[0].squeeze())